In [3]:
import gpflow
import tensorflow as tf
from gpflow.kernels.stationaries import SquaredExponential, Kernel, Matern52
from tensorflow import Tensor
import numpy as np

In [4]:
# old stuff

def old_k_grad_se(inducing_variable_ny, inducing_variable_Zy, kernel):
    funcs2 = tf.range(inducing_variable_Zy.shape[0])
    norm = 1 / (kernel.lengthscales ** 2)
    partial_derivative = tf.map_fn(
        lambda j: (inducing_variable_ny - inducing_variable_Zy[j]),
        funcs2, fn_output_signature=tf.float64, parallel_iterations=8)
    return norm * partial_derivative

def old_kernel_derivative(
        inducing_location_ny: Tensor,
        inducing_location_Zy: Tensor,
        kernel: Kernel,
):
    iterator = tf.range(inducing_location_ny.shape[0])
    block = tf.map_fn(lambda i: old_k_grad_se(inducing_location_ny[i], inducing_location_Zy, kernel),
                      iterator, fn_output_signature=tf.float64, parallel_iterations=8)

    return block

def initialize_Z(num_latent_gps, num_inducing):
    Z = tf.convert_to_tensor(np.array(
        [np.full(num_latent_gps, i) for i in np.linspace(0.1, 0.9, num_inducing)], dtype=np.float64))
    return Z

In [55]:
dof = 1
from gpflow.base import default_float

start = tf.cast((tf.fill((1, dof), 0.)), dtype=default_float())
end = tf.cast((tf.fill((1, dof), 1.)), dtype=default_float())
Z = initialize_Z(dof, 2)
ny = tf.concat([start, end], axis=0)
Zy = tf.concat([ny, Z], axis=0)

In [56]:
Zy.shape

TensorShape([4, 1])

In [57]:
old_kernel_derivative(ny, Zy, SquaredExponential())

<tf.Tensor: shape=(2, 4, 1), dtype=float64, numpy=
array([[[ 0. ],
        [-1. ],
        [-0.1],
        [-0.9]],

       [[ 1. ],
        [ 0. ],
        [ 0.9],
        [ 0.1]]])>

In [8]:
def new_kernel_derivative(ny, Zy, kernel):
    lengthscale = kernel.lengthscales
    block = new_k_grad_se(ny, Zy, lengthscale)
    return block

def new_k_grad_se(ny, Zy, lengthscale):
    norm = 1 / (lengthscale ** 2)
    partial_derivative = norm * tf.subtract(tf.expand_dims(ny, 1), Zy)
    return partial_derivative


In [9]:
new_kernel_derivative(ny, Zy, SquaredExponential())

<tf.Tensor: shape=(2, 4, 7), dtype=float64, numpy=
array([[[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [-1. , -1. , -1. , -1. , -1. , -1. , -1. ],
        [-0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1],
        [-0.9, -0.9, -0.9, -0.9, -0.9, -0.9, -0.9]],

       [[ 1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0.9,  0.9,  0.9,  0.9,  0.9,  0.9,  0.9],
        [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1]]])>

In [12]:
new_kernel_derivative(Zy, Zy, SquaredExponential())

<tf.Tensor: shape=(4, 4, 7), dtype=float64, numpy=
array([[[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [-1. , -1. , -1. , -1. , -1. , -1. , -1. ],
        [-0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1],
        [-0.9, -0.9, -0.9, -0.9, -0.9, -0.9, -0.9]],

       [[ 1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0.9,  0.9,  0.9,  0.9,  0.9,  0.9,  0.9],
        [ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1]],

       [[ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1],
        [-0.9, -0.9, -0.9, -0.9, -0.9, -0.9, -0.9],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [-0.8, -0.8, -0.8, -0.8, -0.8, -0.8, -0.8]],

       [[ 0.9,  0.9,  0.9,  0.9,  0.9,  0.9,  0.9],
        [-0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1],
        [ 0.8,  0.8,  0.8,  0.8,  0.8,  0.8,  0.8],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]]])>

In [30]:
def k_grad_grad_se_fallback(
        inducing_location_ny: Tensor,
        kernel: SquaredExponential,
):
    norm = kernel.lengthscales ** 2
    inducing_diff = tf.expand_dims(inducing_location_ny, 1) - tf.expand_dims(inducing_location_ny, 0)
    second_derivative = norm - inducing_diff ** 2
    return second_derivative / kernel.lengthscales ** 4

In [71]:

@tf.function
def kernel_derivative(ny, Zy, kernel):

    block_original = kernel(Zy)[..., None]
    print(block_original.shape)
    ny_block = k_grad_grad_se_fallback(ny, kernel) * block_original[:ny.shape[0], :ny.shape[0]]
    print(ny_block.shape)
    nyZy_block = new_kernel_derivative(ny, Zy[ny.shape[0]:], kernel) * block_original[:ny.shape[0], ny.shape[0]:]
    print(nyZy_block.shape)
    upper_block = tf.concat([ny_block, nyZy_block], axis=1)
    lower_block = tf.concat([-tf.transpose(nyZy_block, perm=(1, 0, 2)), block_original[ny.shape[0]:, ny.shape[0]:]], axis=1)
    return tf.concat([upper_block, lower_block], axis=0)

In [76]:
aux = kernel_derivative(ny, Zy, SquaredExponential(lengthscales=.5))

(4, 4, 1)
(2, 2, 1)
(2, 2, 1)


In [77]:
tf.transpose(aux, perm=(2, 0 ,1))

<tf.Tensor: shape=(1, 4, 4), dtype=float64, numpy=
array([[[ 4.        , -1.6240234 , -0.39207947, -0.71243532],
        [-1.6240234 ,  4.        ,  0.71243532,  0.39207947],
        [ 0.39207947, -0.71243532,  1.        ,  0.2780373 ],
        [ 0.71243532, -0.39207947,  0.2780373 ,  1.        ]]])>

In [83]:
kk = SquaredExponential(lengthscales=.5)
# tf.print(kk)

In [82]:
ny

<tf.Tensor: shape=(2, 1), dtype=float64, numpy=
array([[0.],
       [1.]])>

In [84]:
(ny[0] - ny[1])/(0.5 ** 2) * kk(ny) 

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[-4.        , -0.54134113],
       [-0.54134113, -4.        ]])>

In [87]:
new_kernel_derivative(ny, ny, kk) * kk(ny)

<tf.Tensor: shape=(2, 2, 2), dtype=float64, numpy=
array([[[ 0.        ,  0.        ],
        [-0.54134113, -4.        ]],

       [[ 4.        ,  0.54134113],
        [ 0.        ,  0.        ]]])>